<a href="https://colab.research.google.com/github/RJuro/ttt/blob/master/journals_stuff_ike.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install polyfuzz -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 14.6 MB/s eta 0:00:00


In [ ]:
from polyfuzz import PolyFuzz
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
all_journals = pd.read_excel('/content/MostUsedSeriesAllGroups.xlsx', header=None)

In [ ]:
all_journals.columns = ['journal','group']

In [ ]:
all_journals = all_journals.dropna()

In [ ]:
all_titles = all_journals.journal.tolist()

In [ ]:
ajg = pd.read_excel('/content/AJG list.xlsx')

In [ ]:
ajg.head()

,ISSN,Field,Jourmal Title,Publisher Name,AJG 2021,AJG 2018,AJG 2015,ABS 2010,Journal Citation Reports rank,SNIP rank,SJR rank,Cite score rank
0,1558-7967,ACCOUNT,Accounting Review,American Accounting Association,4*,4*,4*,4.0,4.0,3.0,3.0,4.0
1,1873-6289,ACCOUNT,"Accounting, Organizations and Society",Elsevier,4*,4*,4*,4.0,5.0,5.0,6.0,5.0
2,1879-1980,ACCOUNT,Journal of Accounting and Economics,Elsevier,4*,4*,4*,4.0,1.0,1.0,2.0,1.0
3,1475-679X,ACCOUNT,Journal of Accounting Research,Wiley-Blackwell,4*,4*,4*,4.0,2.0,2.0,1.0,2.0
4,1911-3846,ACCOUNT,Contemporary Accounting Research,Wiley-Blackwell,4,4,4,3.0,6.0,6.0,4.0,7.0


In [ ]:
ajg_title = ajg['Jourmal Title']

In [ ]:
ajg_title

0                                      Accounting Review
1                  Accounting, Organizations and Society
2                    Journal of Accounting and Economics
3                         Journal of Accounting Research
4                       Contemporary Accounting Research
                              ...                       
1693    Journal of Strategic Contracting and Negotiation
1694                  Journal of Strategy and Management
1695                                          M@n@gement
1696                         Management Review Quarterly
1697                             Strategy and Leadership
Name: Jourmal Title, Length: 1698, dtype: object

In [ ]:
model = PolyFuzz("TF-IDF").match(all_titles, ajg_title.tolist())

In [ ]:
model.get_matches().sort_values('Similarity', ascending=False).reset_index().to_csv('matches_ike.csv')

In [ ]:
matched_journals = model.get_matches().sort_values('Similarity', ascending=False).query("Similarity >= 0.9")

In [ ]:
matched_journals

,From,To,Similarity
471,Journal of Business and Economics,Journal of Economics and Business,1.000
189,Technological Forecasting and Social Change,Technological Forecasting and Social Change,1.000
170,Technovation,Technovation,1.000
174,International Journal of Entrepreneurship and ...,International Journal of Entrepreneurship and ...,1.000
175,Journal of Business Strategy,Journal of Business Strategy,1.000
...,...,...,...
207,Economics and Business Review,Economic and Business Review,0.973
127,The European Journal of Finance,European Journal of Finance,0.961
26,Management Control,Journal of Management Control,0.960
267,Journal of Family Issues,Journal of Family and Economic Issues,0.932


In [ ]:
merged1 = all_journals.merge(matched_journals[['From','To']], how='inner', left_on='journal', right_on='From')

In [ ]:
merged2 = merged1.merge(ajg, left_on='To', right_on='Jourmal Title', how='inner')

In [ ]:
merged2['group'].value_counts(normalize=True)

The IKE Research Group                                   0.261456
International Business Research Group                    0.231806
Accounting Research Group                                0.153639
The Strategy, Organization and Management Group          0.140162
Marketing and Market Processes Research Group            0.115903
Macroeconomic Methodology, Theory and Economic Policy    0.056604
FINANCE                                                  0.040431
Name: group, dtype: float64

In [ ]:
merged2[merged2['AJG 2021']!=1]['group'].value_counts(normalize=True)

The IKE Research Group                                   0.278351
International Business Research Group                    0.199313
Accounting Research Group                                0.164948
The Strategy, Organization and Management Group          0.140893
Marketing and Market Processes Research Group            0.120275
FINANCE                                                  0.048110
Macroeconomic Methodology, Theory and Economic Policy    0.048110
Name: group, dtype: float64

In [ ]:
merged2.groupby('group')['AJG 2021'].value_counts(normalize=True)

group                                                  AJG 2021
Accounting Research Group                              2           0.421053
                                                       3           0.315789
                                                       1           0.157895
                                                       4*          0.070175
                                                       4           0.035088
FINANCE                                                2           0.533333
                                                       3           0.333333
                                                       1           0.066667
                                                       4           0.066667
International Business Research Group                  3           0.383721
                                                       1           0.325581
                                                       2           0.186047
                        

In [49]:
merged2.groupby('group')['AJG 2021'].value_counts(normalize=False)

group                                                  AJG 2021
Accounting Research Group                              2           24
                                                       3           18
                                                       1            9
                                                       4*           4
                                                       4            2
FINANCE                                                2            8
                                                       3            5
                                                       1            1
                                                       4            1
International Business Research Group                  3           33
                                                       1           28
                                                       2           16
                                                       4*           6
                          